In [1]:
include("./perov_functions.jl")
using Plots
using BenchmarkTools
using Distributions
using LaTeXStrings
using Measures

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9099953095449962093\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4031704100306815350\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9099953095449962093\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
n=2;nk=100;
system_original=make_symstem(n);
system=copy(system_original);

In [ ]:
function e1(system)
    axis=rand(Uniform(-1.,1.),3)
    theta=rand(Uniform(0,2π))
    pos=rand(1:size(system)[1],3)
    s,s1=get_rotation_energy_1(system,1,1/250,pos,axis,theta);
    if s<0
        system=s1
        end;
    e=get_total_energy_mag(system,1,1/150)
    return e
    end;

In [14]:
function run_sum(n,system,g1,g2)
    e=[]
    a1=[]
    a2=[]
    a3=[]
    mag_val=[]
    for i in 1:n
        axis=rand(Uniform(-1.,1.),3)
        theta=rand(Uniform(0,2π))
        pos=rand(1:size(system)[1],3)
        s,s1=get_rotation_energy_1(system,g1,g2,pos,axis,theta);
        if s<0
            system=s1
            end;
        if i%100==0
            vals=get_total_energy_mag(system,g1,g2)
            energy=vals[1]
            angles=vals[2].^2
            push!(e,energy)
            push!(a1,angles[1])
            push!(a2,angles[2])
            push!(a3,angles[3])
            push!(mag_val,norm(vals[2]))
            end;
        end;
    return e,a1,a2,a3,mag_val,system
    end;

In [45]:
system=copy(system_original);
@time e,a1,a2,a3,mag_val,system=run_sum(60000,system,1,1/950);

 29.081049 seconds (309.01 M allocations: 34.007 GiB, 25.15% gc time)


In [48]:
function animplots(i)
gr()
pl1=plot((1:i)*100,e[1:i],ylabel="energy",xlabel="iterations",color=RGB(226/255, 194/255, 144/255),label="",linewidth=2)
pl2=plot((1:i)*100,a1[1:i],label=L"$s_x$",xlabel="iterations",ylabel="component",ylim=(-0.01,1.1),linewidth=1.5)
plot!(pl2,(1:i)*100,a2[1:i],label=L"$s_y$",linewidth=1.5)
plot!(pl2,(1:i)*100,a3[1:i],label=L"$s_z$",linewidth=1.5)
pl3=plot((1:i)*100,mag_val[1:i],ylabel=L"$M$",xlabel="iterations",color=RGB(202/255, 255/255, 191/255),linewidth=2,label="",ylim=(-0.01,1.1))
hline!(pl3,[1],linestyle=:dash,color="grey",label="")
hline!(pl2,[1/3],linestyle=:dash,color="grey",label="")
plot(pl1,pl3,pl2,size=(1200,400),dpi=100,bottom_margin=5mm,layout=@layout [a b c])
end;

In [ ]:
anim = @animate for i ∈ 1:length(e)
    animplots(i)
end
gif(anim, "animations/MC_3.gif", fps = 17)

In [4]:
cnt=0
for i in 1:100
    axis=rand(Uniform(-1.,1.),3)
    theta=rand(Uniform(0,2π))
    pos=rand(1:size(system)[1],3)
    dE,s1=get_rotation_energy_1(system,1,1/250,pos,axis,theta);
    T=.2
    if dE < 0 || rand() < exp(-dE/T)
        cnt+=1
    end
end
print(cnt)

19